# 여러 모듈을 쉽게 조합할 수 있는 Chains

## LLMChain을 사용해 여러 모듈을 통합하는 방법

In [34]:
from langchain import LLMChain, PromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model = 'gpt-3.5-turbo'
)

prompt = PromptTemplate(
    template="{product}는 어느 회사에서 개발한 제품인가요?",
    input_variables=[
        "product"
    ]
)

chain = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True
)

In [35]:
result = chain.predict(product='iPhont')



> Entering new LLMChain chain...
Prompt after formatting:
iPhont는 어느 회사에서 개발한 제품인가요?

> Finished chain.


In [36]:
print(result)

iPhont는 Apple이 개발한 제품입니다.


## ConversationChain으로 기억을 가진 애플리케이션 개발이 쉬워진다.

In [33]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI()

memory = ConversationBufferMemory(return_messages=True)

chain = ConversationBufferMemory(
    memory=memory,
    llm=chat
)

# 특정 기능에 특화된 Chain

## 특정 URL에 접속해 정보를 얻게 하는 방법

In [81]:
url = 'https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json'

In [82]:
url

'https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json'

In [88]:
from langchain.chains import LLMChain, LLMRequestsChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI()

prompt = PromptTemplate(
    input_variables=['query', 'requests_result'],
    template=""" 아래의 문장을 바탕으로 질문에 답해 주세요.
    문장 : {requests_result}
    질문 : {query} 
    """
)

llm_chain = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True
)

chain = LLMRequestsChain(
    llm_chain=llm_chain,
)

In [91]:
chain({
    "query":"도쿄의 날씨를 알려주세요",
    "url":'https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json'
})



> Entering new LLMChain chain...
Prompt after formatting:
 아래의 문장을 바탕으로 질문에 답해 주세요.
    문장 : {"publishingOffice":"気象庁","reportDatetime":"2024-05-26T04:38:00+09:00","targetArea":"東京都","headlineText":"","text":"　本州付近は高気圧に覆われていますが、関東甲信地方は湿った空気の影響を受けています。\n\n　東京地方は、曇りとなっています。\n\n　２６日は、高気圧に覆われますが、湿った空気の影響を受ける見込みです。このため、曇りで昼過ぎから晴れるでしょう。\n\n　２７日は、日本海を前線が東進し、湿った空気の影響を受ける見込みです。このため、曇りで、昼過ぎから夕方は雨の降る所があるでしょう。\n\n【関東甲信地方】\n　関東甲信地方は、曇りで、弱い雨の降っている所があります。\n\n　２６日は、高気圧に覆われますが、湿った空気の影響を受ける見込みです。このため、曇りや晴れで、明け方まで雨の降る所があるでしょう。\n\n　２７日は、日本海を前線が東進し、湿った空気の影響を受ける見込みです。このため、曇りや晴れで、昼前から雨の降る所があるでしょう。\n\n　関東地方と伊豆諸島の海上では、２６日は波がやや高く、２７日は波が高いでしょう。船舶は高波に注意してください。"}
    질문 : 도쿄의 날씨를 알려주세요 
    

> Finished chain.


{'query': '도쿄의 날씨를 알려주세요',
 'url': 'https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json',
 'output': '답변 : 도쿄 지역은 26일은 曇り(흐림)이며, 낮에 햇빛이 나타날 것으로 예상됩니다. 27일은 曇り(흐림)이며, 낮에 비가 오는 곳이 있을 것으로 예상됩니다.'}

# Chains 자체 정리하기

## Chains 자체를 순서대로 실행하는 SimpleSequentialChain

In [95]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(model='gpt-3.5-turbo')

write_article_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate(
        template="{input}에 대한 기사를 써주세요.",
        input_variables=['input']
    )
)

translate_chain =  LLMChain(
    llm=chat,
    prompt=PromptTemplate(
        template="다음 문장을 영어로 번역해 주세요.\n{input}",
        input_variables=['input']
    )
)

sequential_chain = SimpleSequentialChain(
    chains=[
        write_article_chain,
        translate_chain,
    ]
)

In [96]:
result = sequential_chain("일렉트릭 기타 선택 방법")
print(result)

{'input': '일렉트릭 기타 선택 방법', 'output': 'Choosing an electric guitar is a very important decision. With various brands, models, and styles available, it is crucial to select the right guitar. In this article, we will guide you on how to choose an electric guitar.\n\nThe first factor to consider is your budget. It is important to choose a guitar that fits within your budget. Setting a budget and looking for a guitar within that range is advisable.\n\nSecondly, you should consider the sound and style you desire. Think about the genre of music you want to play and the sound you are looking for. Each guitar has its own unique sound and various styles.\n\nThirdly, you should consider the materials of the guitar. The materials such as wood, pickups, and bridge impact the sound and playability of the guitar. Choosing the right material for you is important.\n\nFourthly, consider the size and weight of the guitar. It is important to choose a guitar that fits well in your hands and is comfortable 